# 8. 판다스 자료형

## 8.1. 자료형 다루기

### 8.1.1. 자료형 변환하기

In [1]:
import pandas as pd
import seaborn as sns

tips = sns.load_dataset("tips") #seaborn library의 데이터 집합

In [2]:
# astype method : Cast a pandas object to a specified dtype dtype.
tips['sex_str'] = tips['sex'].astype(str) # category type을 string type으로 변환

print(tips[['sex','sex_str']].head())

print()

print(tips.dtypes)

# 출력값은 똑같지만, type은 다름.

      sex sex_str
0  Female  Female
1    Male    Male
2    Male    Male
3    Male    Male
4  Female  Female

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


In [3]:
tips['total_bill'] = tips['total_bill'].astype(str) # float64 type을 string type으로 변환

print(tips.dtypes)

total_bill      object
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


In [4]:
tips['total_bill'] = tips['total_bill'].astype(float) # string type을 float type으로 재변환

print(tips.dtypes)

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


### 8.2.2. 잘못 입력한 데이터 처리하기

In [ ]:
tips_sub_miss = tips.head(10)
tips_sub_miss.loc[[1,3,5,7], 'total_bill'] = 'missing'

print(tips_sub_miss)
# Pandas에서 데이터프레임의 특정 열에 dtype이 서로 다른 값을 할당하려고 해서 마지막 오류 발생

  total_bill   tip     sex smoker  day    time  size sex_str
0      16.99  1.01  Female     No  Sun  Dinner     2  Female
1    missing  1.66    Male     No  Sun  Dinner     3    Male
2      21.01  3.50    Male     No  Sun  Dinner     3    Male
3    missing  3.31    Male     No  Sun  Dinner     2    Male
4      24.59  3.61  Female     No  Sun  Dinner     4  Female
5    missing  4.71    Male     No  Sun  Dinner     4    Male
6       8.77  2.00    Male     No  Sun  Dinner     2    Male
7    missing  3.12    Male     No  Sun  Dinner     4    Male
8      15.04  1.96    Male     No  Sun  Dinner     2    Male
9      14.78  3.23    Male     No  Sun  Dinner     2    Male


<ipython-input-7-432833f2e6fd>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'missing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tips_sub_miss.loc[[1,3,5,7], 'total_bill'] = 'missing'


In [ ]:
# total_bill열이 float형이 아니라 문자열임. ∵ 'missing'
print(tips_sub_miss.dtypes)

total_bill      object
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


In [ ]:
# tips_sub_miss['total_bill'].astype(float) : 오류 발생 ← 'missing'이라는 문자열을 실수로 변환 불가
# pd.to_numeric(tips_sub_miss['total_bill']) : 오류 발생 ←  마찬가지로 'missing'이라는 문자열을 실수로 변환 불가

# to_numeric method의 <error parameter>를 사용
# 1. raise (default) : 숫자로 변환할 수 없는 값이 있으면 오류 발생
# 2. coerce : 숫자로 변환할 수 없는 값을 누락값으로 지정
# 3. ignore : 아무 작업도 하지 않음.

tips_sub_miss['total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'],
                                            errors='coerce',
                                            downcast='float') # float64가 아닌 float32(표현 실수 범위 작음)로 변환 → 메모리 1/2배
print(tips_sub_miss.dtypes)

'''
오류 안뜨는 코드
tips_sub_miss.loc[:, 'total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'],
                                                   errors='coerce',
                                                   downcast='float')
'''

total_bill     float32
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
sex_str         object
dtype: object


<ipython-input-9-60e7a1481d0d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tips_sub_miss['total_bill'] = pd.to_numeric(tips_sub_miss['total_bill'],


위 경고 뜨는 이유   

*  Copy-on-Write : \\
Pandas 3.0 버전에서 도입될 예정 \\
어떤 데이터를 수정하려고 할 때, 원본 데이터를 바로 수정하는 것이 아니라 수정된이터의 복사본을 만들어서 작업을 수행 \\
데이터 무결성을 유지하고 예기치 않은 변경을 방지하는 데 유용 \\

 \\
*   Chained Indexing : \\
Pandas에서 DataFrame이나 Series의 데이터를 접근하거나 수정할 때, 두 개 이상의 인덱싱 연산을 연결해서 사용하는 경우 \\
Chained indexing을 수행할 때 중간 단계의 결과가 원본 데이터의 "뷰(view)"인지 "복사본(copy)"인지 확실히 알 수 없기 때문에 발생 \\

 \\
* Pandas 3.0에서는 Copy-on-Write가 기본 동작으로 설정되어 'SettingWithCopyWarning' 경고 뜨지 않음.


## 8.2. 카테고리 자료형


*   용량과 속도 면에서 매우 효율적
*   주로 동일한 문자열이 반복되어 데이터를 구성하는 경우 사용



In [ ]:
tips['sex'] = tips['sex'].astype('str')
print(tips.info())
# 문자열인 경우 용량 10.8+KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    object  
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
 7   sex_str     244 non-null    object  
dtypes: category(3), float64(2), int64(1), object(2)
memory usage: 10.8+ KB
None


In [ ]:
tips['sex'] = tips['sex'].astype('category')
print(tips.info())
# 카테고리인 경우 용량 90.3+KB → 훨 줄어듦.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
 7   sex_str     244 non-null    object  
dtypes: category(4), float64(2), int64(1), object(1)
memory usage: 9.3+ KB
None
